In [1]:
import sqlite3
import time
import pandas as pd
import numpy as np
import query
import preprocesser
import datetime
import pandas.tseries.offsets as of  # for timeoffset

In [2]:
#1.종목명
shcode_list = query.query_shcode_list_today()

In [3]:
#2.target from csvs
df_bf = pd.read_csv("bf_target_day_using_weekly_v3.csv",encoding='CP949')
df_af = pd.read_csv("af_target_day_using_weekly_v3.csv",encoding='CP949')
del df_bf['Unnamed: 0']
del df_af['Unnamed: 0']

In [4]:
df2_bf = df_bf[['index', 'shcode','month_ma5_score', 'high_month', 
               'for_net_buy_sum', 'com_net_buy_sum','per_net_buy_sum', 
               'week_ma5_score', 'week_ma20_score',
               'week_golden_cross', 'week_dead_cross', 'week_up', 'week_down']].drop_duplicates()

In [5]:
df_bfaf = pd.merge(df_af, df2_bf, how = 'left', on = ['index', 'shcode'])

In [6]:
uplist = query.query_up_distinct()
tmlist = query.query_tm_distinct()

In [7]:
df_bfaf = pd.merge(df_bfaf, uplist[['shcode','upcode', 'upname']], how = 'left', on = ['shcode'])
df_bfaf = pd.merge(df_bfaf, tmlist[['shcode','tmcode', 'tmname']], how = 'left', on = ['shcode'])

In [8]:
'''
df_bfaf[['index', 'shcode', 'shname', 'target_m', 'date', 'close_ratio', 'open_',
        'low', 'high', 'close', 'min_target', 'max_target', 'range_position',
        'volume', 'volume_ratio', 'spot_range', 'inrange','volume_default', 'squeezed',
        'obv', 'upcode', 'upname', 'tmcode', 'tmname',
        'month_ma5_score', 'high_month','for_net_buy_sum', 'com_net_buy_sum', 'per_net_buy_sum',
        'week_ma5_score', 'week_ma20_score', 'week_golden_cross',
        'week_dead_cross', 'week_up', 'week_down', 'for_net_buy', 'com_net_buy', 'per_net_buy',
        'close_ma5', 'close_ma10','close_ma20', 'close_ma60', 'close_ma120']].to_excel("bfaf_target_day_using_weekly_v4.xlsx",encoding = 'ms949')
'''

In [9]:
df_bfaf = df_bfaf[['index', 'shcode', 'shname', 'target_m', 'date', 'close_ratio', 'open_',
                    'low', 'high', 'close', 'min_target', 'max_target', 'range_position',
                    'volume', 'volume_ratio', 'spot_range', 'inrange','volume_default', 'squeezed',
                    'obv', 'upcode', 'upname', 'tmcode', 'tmname',
                    'month_ma5_score', 'high_month','for_net_buy_sum', 'com_net_buy_sum', 'per_net_buy_sum',
                    'week_ma5_score', 'week_ma20_score', 'week_golden_cross',
                    'week_dead_cross', 'week_up', 'week_down', 'for_net_buy', 'com_net_buy', 'per_net_buy',
                    'close_ma5', 'close_ma10','close_ma20', 'close_ma60', 'close_ma120']]

In [4]:
#uplist[uplist['shcode']=='000080']
#uplist[uplist['shcode']=='000080'].groupby(['shcode']).count()

In [3]:
#tmlist[tmlist['shcode']=='000080']
#tmlist[tmlist['shcode']=='000080'].groupby(['shcode']).count()

In [ ]:
#df_bfaf2 = df_bfaf[['index','shcode']].drop_duplicates()

In [2]:
#1.종목명
shcode_list = query.query_shcode_list_today()

#2.target from csvs
df = pd.read_csv("high_ratio_0.05_volume_money_20000000000_v3.csv", dtype ={'shcode':str})

#3. target index 추출
fin_result = pd.DataFrame([],columns = ['target_day','shcode'])

for i,v in df['index'].items():
    shcode = df['shcode'].iloc[i].zfill(6)
    target_date = v.split(" ")
    target_day_list = pd.to_datetime(target_date, format = "%Y-%m-%d")
    interval_end_timestamp = target_day_list + datetime.timedelta(weeks=6)
    result = pd.DataFrame(dict(target_day = target_day_list.strftime("%Y%m%d"),
                              interval_end = interval_end_timestamp.strftime("%Y%m%d"))).reset_index()
    del result['index']
    result['shcode'] = shcode
    fin_result = fin_result.append(result)
    
#4. make target month
fin_result['target_m'] = fin_result['target_day'].str.slice(stop = 6)
fin_result = pd.merge(fin_result, shcode_list, how = 'inner', on = 'shcode')

#5. target day 확인
target_day_list = pd.to_datetime(fin_result['target_day'], format = "%Y-%m-%d")

#(1) for jupo
#interval calculate before 12 month 
init_jupo_timestamp = target_day_list + of.MonthEnd(n= -12)
end_jupo_timestamp = target_day_list
fin_result['init_jupo'] = init_jupo_timestamp.apply(lambda x: x.strftime("%Y%m%d"))
fin_result['end_jupo'] = end_jupo_timestamp.apply(lambda x: x.strftime("%Y%m%d"))


fin_result = fin_result[fin_result['target_m']>='201909'].reset_index()
del fin_result['index']
fin_result['index'] = fin_result.index

C:\ProgramData\Anaconda3\envs\py36_32\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [ ]:
pd.merge(df_bfaf2, uplist[['shcode','upcode', 'upname', 'per']], how = 'left', on = ['shcode'])

In [3]:
fin_result.iloc[410:415]

,interval_end,shcode,target_day,target_m,shname,init_jupo,end_jupo,index
410,20191105,078590,20190924,201909,두올산업,20180930,20190924,410
411,20191113,078590,20191002,201910,두올산업,20181031,20191002,411
412,20191216,078860,20191104,201911,아이오케이,20181130,20191104,412
413,20191106,079950,20190925,201909,인베니아,20180930,20190925,413
414,20191227,079960,20191115,201911,동양이엔피,20181130,20191115,414


In [25]:
fin_result.columns

Index(['interval_end', 'shcode', 'target_day', 'target_m', 'shname',
       'init_month', 'end_month', 'index'],
      dtype='object')

In [4]:
# for jupo query
a = query.query_jupo_vol(fin_result['shcode'].iloc[414], fr=fin_result['init_jupo'].iloc[414], to='20191202')
# cumsum index
c = a.groupby(['shcode','date']).sum().cumsum()
# multi index to single index
c = c.reset_index(level=['shcode','date'])
# calculate max volume
c_for_value = c[['samo_vol', 'sec_vol', 'ins_vol', 'tusin_vol', 'bank_vol',
                 'jong_vol', 'fund_vol', 'etcom_vol', 'per_vol', 'for_reg_vol',
                 'for_noreg_vol', 'nat_no_vol', 'com_vol', 'for_vol', 'etc_vol']]
c['max_vol'] = c_for_value.max(axis=1)
c['max_jupo'] = c_for_value.idxmax(axis=1)
c = c.assign(max3_sum=np.sum(np.partition(c_for_value.values, -3)[:, -3:], 1))



In [13]:
fin_result['shcode'].iloc[0]

'000230'

In [15]:
b = query.query_finance(fin_result['shcode'].iloc[414])

In [16]:
b

,date,shcode,close,volume,volume_money,high_52,high_52_day,low_52,low_52_day,rotate_ratio,burn_ratio,per,total_vol,ev_ratio,vol_scale,market_value
0,20191202,079960,16800,183482,3165,19500,20191119,9000,20190107,15.70,2.33,23.06,7860,40,00001,1368


In [ ]:
'최고가_52','최고가일_52','최저가_52','최저가일_52','소진율','회전율','PER'
'상장주식수_천','증거금율','수량단위',